In [7]:
import pandas as pd
import os

# Wrangle anime data

In [8]:
# Load data
animes = pd.read_csv("data/animes.csv")
AnimeList = pd.read_csv("data/AnimeList.csv")

In [9]:
AnimeList = AnimeList[["anime_id", "type", "source", "producer", "licensor", "studio"]]
animes = animes.merge(AnimeList, how = "left", left_on = "uid", right_on = "anime_id")
animes = animes[[
    "uid", 
    "title", 
    "synopsis", 
    "genre", 
    "type",
    "episodes", 
    "source", 
    "producer", 
    "licensor", 
    "studio"
]]
animes.head()

,uid,title,synopsis,genre,type,episodes,source,producer,licensor,studio
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",TV,25.0,Manga,"TOHO animation, Shueisha",Sentai Filmworks,Production I.G
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",TV,22.0,Manga,"Aniplex, Dentsu, Kodansha, Fuji TV, Kyoraku In...",Aniplex of America,A-1 Pictures
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",TV,13.0,Web manga,"Media Factory, AT-X, Takeshobo, Sony Music Com...",Sentai Filmworks,Kinema Citrus
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",TV,64.0,Manga,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",Movie,1.0,Light novel,"Aniplex, Kodansha",NaN,Shaft
